__качаем объявления о продаже домов с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
# avito_path = 'sevastopol/kvartiry/prodam'

# place = 'sevastopol'
# place = 'bahchisaray'
place = 'sudak'

avito_path = place+'/doma_dachi_kottedzhi/prodam'

raw_data_path = 'data/raw/house/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-26 11:22:38 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-26 11:22:38 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-26 11:22:40 | AdsListParser: downloader init
[INFO    ] 2022-09-26 11:22:40 | AvitoParser: init
[INFO    ] 2022-09-26 11:22:47 | AvitoParser: 3 pages for read
[INFO    ] 2022-09-26 11:22:47 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-26 11:22:54 | AdsListParser: read page 1
[INFO    ] 2022-09-26 11:22:58 | AdsListParser: read page 2
[INFO    ] 2022-09-26 11:23:01 | AdsListParser: read page 3
[INFO    ] 2022-09-26 11:23:01 | AdsListParser: last page detected
[INFO    ] 2022-09-26 11:23:01 | DownloaderSeleniumFirefox: close virtual browser
122
CPU times: user 1.37 s, sys: 174 ms, total: 1.55 s
Wall time: 23.7 s


In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

122
122


In [8]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
49,2318367244,"Объявление «Дом 60,5 м² на участке 6 сот.» 40 фотографий",4200000,,"с. Миндальное, СНТ Солнечная Долина, Арбатская ул., 17","Бархатный Сезон- Море +24… Предлагаю вам стать владельцем(Не Агентство- не могу поменять) современного Домика в Крыму (по стоимости 1к. Кв) в пригороде прекрасного курортного г. Судак, в с. Миндальное. Домик Новый (въехали в июне 2021г.) 60,5 кв. М.-по док",1,2022-09-26 11:23:01.176417
114,2405525781,Объявление «Дом 223 м² на участке 6 сот.» 6 фотографий,23000000,,"ул. Мамеди, 28","Продаётся дом, на 6 сотках в солнечном городе Судак с уникальным климатом. Дом построен в тихом месте в 250 м от городской транспортной остановки, в 2 км до моря. Удобная транспортная развязка с выездом в Симферополь, мыс Меганом, Алушта. Дом капитального",3,2022-09-26 11:23:01.176417


In [9]:
df['place'] = place

In [10]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw_{place}.xlsx',index=False)